# Understanding the problem statement and business case
* In this hand-on project, we will train 3 Machine Learning algorithms namely Multiple Linear Regression, Random Forest Regression and XGBoost to predict the price of used cars.
* INPUTS(FEATURES): Make, Model, Type, Origin, Drivetrain, Invoice, EngineSize, Cylinders, Horesepower, MPG_City, MPG_Highway, Weight, Wheelbase and Lenght.
* OUTPUT: MSRP (price)
* This project can be used by car dealerships to predict used car prices and understand key factors that contribute to used car prices.

# Libraries/dataset import

In [ ]:
!pip install xgboost
!pip install wordcloud
!pip install plotly

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from math import sqrt

Import dataset from github

In [ ]:
!wget https://raw.githubusercontent.com/VM-137/used-car-price-prediction/main/CARS.csv

In [ ]:
car_df = pd.read_csv('/content/CARS.csv') # create pandas datafrme "car_df"

In [ ]:
car_df

Display features columns

# Data Preprocessing (1)
* Check features
* Check shape
* Check missing values
* Check data types

### Features

In [ ]:
car_df.columns #Display all column names

Check the shape and missing values

### Shape

In [ ]:
car_df.shape # the shape of the dataframe "car_df"

### Missing values

In [ ]:
car_df.isna().sum() # sum missing values of all columns

There are just 2 missing values, we will drop that two rows

In [ ]:
car_df = car_df.dropna() # drop the two rows with missing values in "Cylinders" column.

In [ ]:
car_df.info() # show information of the columns

### Data types
* Convert 'MSRP' and 'Invoice' datatype to integer, we need to remove '$' str and comma ',' from these 2 columns

In [ ]:
print(car_df[['MSRP', 'Invoice']].dtypes) # check datatypes of 'MSRP & 'Invoice'


In [ ]:
# convert to string
car_df['MSRP'] = car_df['MSRP'].astype(str)
car_df['Invoice'] = car_df['Invoice'].astype(str)


In [ ]:
print(car_df[['MSRP', 'Invoice']].dtypes) # check datatypes of 'MSRP & 'Invoice'

In [ ]:
# remove "$" and "," then convert to integer
car_df['MSRP'] = car_df['MSRP'].str.replace('$', '')
car_df['MSRP'] = car_df['MSRP'].str.replace(',', '')
car_df['MSRP'] = car_df['MSRP'].astype(int)
car_df['Invoice'] = car_df['Invoice'].str.replace('$', '')
car_df['Invoice'] = car_df['Invoice'].str.replace(',', '')
car_df['Invoice'] = car_df['Invoice'].astype(int)

In [ ]:
car_df[['MSRP', 'Invoice']].head(10)

In [ ]:
car_df.info()

# Exploratory Data Analysis (1)
* Dataset summary
* Scatter plots and histograms
* 'Model' words cloud visualization

### Dataset summary
* Coun, mean, std, min, max of each feature

In [ ]:
# Generate descriptive statistics for data
car_df.describe()

In [ ]:
# Print the max value of column 'MSRP'
print('Maximum price for used car: ', car_df['MSRP'].max())

In [ ]:
#Print the minimum value of column 'MSRP'
print('Minimum price for used car: ', car_df['MSRP'].min())

### Visualization

* Scatter plots for relationship and histograms for univariate distributions

In [ ]:
# To plot multiple pairwise bivariate distributions in a dataset,
# but the diagonal plots show the density of the points
sns.pairplot(data=car_df)

What's going on?
It seems that there is a linear trend between 'Horsepower' and 'MSRP' (the price), probably it have a direct impact on the price.
We can analyze other features and see that there seems to be an inverse relation between 'MPG_' and 'Horsepower' or 'MPG_'and 'Enginesize' which makes sense as far as we are talking about normal cars.

* Let's view unique makes of the cars

In [ ]:
# Get all the unique value from the column of 'Make'
car_df.Make.unique()

In [ ]:
# Create a histogram showing the count number of each manufacturer
fig = px.histogram(car_df, x = 'Make',
                   labels = {'Make': 'Manufacturer'},
                   title = 'Make OF THE CAR',
                   color_discrete_sequence = ['maroon']
                   )
fig.show()

* Let's view various types of cars

In [ ]:
# List all the unique values of column 'Type'
car_df.Type.unique()

In [ ]:
# Create a histogram showing the count number of each 'Type' of car
fig = px.histogram(car_df, x='Type',
                   labels = {'Type':'Type'},
                   title = 'TYPE OF CAR',
                   color_discrete_sequence = ['brown']
                   )
fig.show()

* Let's view the location of the car sales

In [ ]:
# List all the unique values of column 'Origin'
car_df.Origin.unique()

In [ ]:
# Create a histogram showing the count number of each 'Origin'
fig = px.histogram(car_df, x='Origin',
                   labels = {'Origin':'Origin'},
                   title = 'LOCATION OF THE CAR SALES',
                   color_discrete_sequence= ['brown']
                   )
fig.show()

* Let's view the drivetrain of the cars

In [ ]:
car_df.DriveTrain.unique()

In [ ]:
# Create a histogram showing the count number of each 'DriveTrain'
fig = px.histogram(car_df, x = 'DriveTrain',
                   labels = {'DriveTrain':'DriveTrain'},
                   title = 'DRIVETRAIN OF THE CAR',
                   color_discrete_sequence = ['BLACK']
                   )
fig.show()

* Plot of the make of the car and its location

In [ ]:
# Create a histogram showing the count number of each 'Manufacturer' and use colors to differentiate the 'Origin'
fig = px.histogram(car_df, x = 'Make',
                   color = 'Origin',
                   labels = {'Make':'Manufacturer'},
                   title = 'MAKE OF THE CAR Vs LOCATION'
                   )
fig.show()

* Plot of the make of the car and its type

In [ ]:
# Create a histogram showing the count number of each 'Make' and use colors to differentiate the 'Type'
fig = px.histogram(car_df, x = 'Make',
                   color = 'Type',
                   labels = {'Make':'Manufacturer'},
                   title = 'MAKE OF THE CAR Vs TYPE'
                   )
fig.show()

* Let's analyze the distribution of words in the 'Model' column using WordCloud generator

In [ ]:
# Create an array with all the values of column 'Model'
text = car_df['Model'].values
# Set up words which are so common that can be ignored in program
stopwords = set(STOPWORDS)

In [ ]:
# WordCloud uses font-size and font-color to visualize the distribution of words
# background_clor sets background,max_words sets the amount of different values,
# Attribute “stopwords” refers to the most frequent words, such as “the” or “and”, that help build ideas but have no meaning in themselves.We usually remove them from documents before any attempt is made to classify them.
wc = WordCloud(background_color='black',
               max_words=2000,
               max_font_size=100,
               random_state=3,
               stopwords=stopwords,
               contour_width=3,
               ).generate(str(text))

In [ ]:
fig = plt.figure(figsize = (25, 15))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

We can visualize the most common words used in the model name.



* Correlation matrix

In [ ]:
# Compute pairwise correlation of columns, excluding NA/null values
car_df.corr()

In [ ]:
# Use heatmap to visualize the correlation of columns
# for a complete guide to heatmap see: https://chartio.com/learn/charts/heatmap-complete-guide/
# Correlation Concepts, Matrix & Heatmap using Seaborn see: https://vitalflux.com/correlation-heatmap-with-seaborn-pandas/
sns.heatmap(car_df.corr(), annot=True)

We can see that the feature with highest correlation to 'MSRP' seems to be 'Horsepower' with 0.83 as we have suspected, the inverse happens with 'MPG_' and 'Enginesize' with negative (~0.7) correlation.

# Data Preprocessing (2)
* Perform One-Hot Encoding for 'Make', 'Model', 'Type', 'Origin' and 'DriveTrain'
* Drop 'Invoice' column as it does not contribute.
* Split the dataset into Train and Test parts


* One-hot encoding

In [ ]:
# Convert categorical variable into dummy/indicator variable
df_dum = pd.get_dummies(car_df,
                        columns = ['Make',
                                   'Model',
                                   'Type',
                                   'Origin',
                                   'DriveTrain']
                        )


In [ ]:
df_dum.head(10)

* Drop 'Invoice' column

In [ ]:
# Drop column 'Invoice' and save as "df_data"
df_data = df_dum.drop(columns=['Invoice'], axis=1)

In [ ]:
df_data.head()

In [ ]:
print('DataFrame shape = ',df_data.shape) # display the dataframe size

Split dependent and independent variables

In [ ]:
# Split column 'MSRP' from the rest of the columns and save in variable X as features and y as target variable
X = df_data.drop(columns=['MSRP'], axis=1)
y = df_data['MSRP']

In [ ]:
X

In [ ]:
y

In [ ]:
# Set 80% of the data as training data set and the rest (20%) as the testing data set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print('X Train dataset shape = ', X_train.shape)
print('y Train dataset shape = ', y_train.shape)
print('X Test dataset shape = ', X_test.shape)
print('y Test dataset shape = ', y_test.shape)

# Model (1)
* Linear Regression

In [ ]:
# Involve linear regression model
L_model = LinearRegression()
# Let training data set to train our linear regression model
L_model.fit(X_train, y_train)

In [ ]:
# Use the testing data set to test the accuracy of the model
# The accuracy_score function compares the true labels with the predicted labels
# and returns the accuracy score as a value between 0 and 1.
accuracy_L_model = L_model.score(X_test, y_test)
print('Score = ', accuracy_L_model)

In [ ]:
from sklearn.svm import SVR  # for regression tasks

# For regression:
svm_model = SVR(kernel='linear')  # Similar to classification, you can change the kernel type

# Training the SVM:
# svm_classifier.fit(X_train, y_train)  # For classification
svm_model.fit(X_train, y_train)   # For regression


In [ ]:
accuracy_svm = svm_model.score(X_test, y_test)  # Compare predicted labels with actual labels
print(f'Accuracy: {accuracy_svm *100:.2f}%')

# Model (2)
* Decision tree
* Random forest

* Decision Tree

In [ ]:
# Involve the decision tree regression model
DecisionTree_model = DecisionTreeRegressor()
# Let the training data set to train our decision tree regression model
DecisionTree_model.fit(X_train, y_train)

In [ ]:
# Use the testing data set to test the accuracy of the predicting model
accuracy_DecisionTree = DecisionTree_model.score(X_test, y_test)
print('Score Decision Tree = ', accuracy_DecisionTree)

* Random forest

In [ ]:
# Invole the random forest regression model
RandomForest_model = RandomForestRegressor(n_estimators=5, max_depth=5)
# Let the training data set to train the random forest regression model
RandomForest_model.fit(X_train, y_train)

In [ ]:
# Use the random forest regression model to test the accuracy of the trained model
accuracy_RandomForest = RandomForest_model.score(X_test, y_test)
print('Score Random Forest = ', accuracy_RandomForest)

# Model (3)
* XGBoost

In [ ]:
# Involve in the XGRegression model
model = XGBRegressor(objective ='reg:squarederror',
                     base_score=0.5,
                     learning_rate=0.15
                     )
# Let the training data set to train this model
model.fit(X_train, y_train)

In [ ]:
# Use the testing data set to test the accuracy of this model
accuracy_XGBoost = model.score(X_test, y_test)
print('Score XGBoost = ', accuracy_XGBoost)

# Model comparison

* Linear regression

In [ ]:
# predict quantities with the finalized regression model by calling the predict() function on the finalized model.
y_predict_linear = L_model.predict(X_test)
# This method is used to plot data and a linear regression model fit
fig = sns.regplot(x=y_predict_linear, y=y_test, color='red', marker='^')
fig.set(title='Linear Regression Model',
        xlabel='Predicted Price of the used cars ($)',
        ylabel = 'Actual price of the used cars ($)'
        )

In [ ]:
# RMSE is an acronym for Root Mean Square Error, which is the square root of value obtained from Mean Square Error function
# he Mean Squared Error (MSE) or Mean Squared Deviation (MSD) of an estimator measures the average of error squares
# Mean Absolute Error calculates the average difference between the calculated values and actual values
# Coefficient of determination also called as R2 score is used to evaluate the performance of a linear regression model.
RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict_linear))))
MSE = mean_squared_error(y_test, y_predict_linear)
MAE = mean_absolute_error(y_test, y_predict_linear)
r2 = r2_score(y_test, y_predict_linear)
print('RMSW =', RMSE,'\nMSE =', MSE, '\nMAE =', MAE, '\nR2 =', r2)

* Random forest

In [ ]:
# predict quantities with the finalized regression model by calling the predict() function on the finalized model.
y_predict_RandomForest = RandomForest_model.predict(X_test)
# This method is used to plot data and a linear regression model fit
fig = sns.regplot(x=y_predict_RandomForest, y=y_test, color='blue', marker='s')
fig.set(title='Random Forest Regression Model',
        xlabel='Predicted Price of the used cars($)',
        ylabel = 'Actual price of the used cars ($)'
        )

In [ ]:
# RMSE is an acronym for Root Mean Square Error, which is the square root of value obtained from Mean Square Error function
# he Mean Squared Error (MSE) or Mean Squared Deviation (MSD) of an estimator measures the average of error squares
# Mean Absolute Error calculates the average difference between the calculated values and actual values
# Coefficient of determination also called as R2 score is used to evaluate the performance of a linear regression model.
RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict_RandomForest))))
MSE = mean_squared_error(y_test, y_predict_RandomForest)
MAE = mean_absolute_error(y_test, y_predict_RandomForest)
r2 = r2_score(y_test, y_predict_RandomForest)
print('RMSW =', RMSE,'\nMSE =', MSE, '\nMAE =', MAE, '\nR2 =', r2)

* XGBoost

In [ ]:
y_predict_XGBoost = model.predict(X_test)
# This method is used to plot data and a linear regression model fit
fig = sns.regplot(x=y_predict_XGBoost, y=y_test, color='blue', marker='s')
fig.set(title='XGBoost Model',
        xlabel='Predicted Price of the used cars($)',
        ylabel = 'Actual price of the used cars ($)'
        )

In [ ]:
# RMSE is an acronym for Root Mean Square Error, which is the square root of value obtained from Mean Square Error function
# he Mean Squared Error (MSE) or Mean Squared Deviation (MSD) of an estimator measures the average of error squares
# Mean Absolute Error calculates the average difference between the calculated values and actual values
# Coefficient of determination also called as R2 score is used to evaluate the performance of a linear regression model.

RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict_XGBoost))))
MSE = mean_squared_error(y_test, y_predict_XGBoost)
MAE = mean_absolute_error(y_test, y_predict_XGBoost)
r2 = r2_score(y_test, y_predict_XGBoost)
print('RMSW =', RMSE,'\nMSE =', MSE, '\nMAE =', MAE, '\nR2 =', r2)